In [ ]:
import cv2
#import numpy as np
import os
from modules_segmentation import *
from mask_corners import gridcorners
import pandas as pd

In [115]:
#set paths
image_folder = "/user/christoph.wald/u15287/big-scratch/02_splitted_data/train_labeled/images_uncropped"
image_files = os.listdir(image_folder)
pest_types = ["BRAIIM", "LIRIBO", "TRIAVA"]
test_folder = "/user/christoph.wald/u15287/insect_pest_detection/image_processing_in_progress/test_segmentation"

#load mask
grown_mask = cv2.imread(
    "/user/christoph.wald/u15287/insect_pest_detection/2_4_image_processing/mask.jpg", 
    cv2.IMREAD_GRAYSCALE
)

inspection = False

In [116]:
#new
results = []

In [128]:
for image_file in image_files[:5]:

    filename= image_file.split(".")[0]
    entry = []
    entry.append(filename)

    #Load image file
    print(f"Loading {image_file}.")
    image = cv2.imread(os.path.join(image_folder, image_file))
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_original.jpg"), image)
    
    #find YST contour of image
    imageYST = find_contour(image)
    imagecorners = find_corners(image, imageYST)
    if len(imagecorners) == 0:
        print(f"No YST found for {image_file}")
        continue
    imagecorners = order_corners(imagecorners)

    #find transformation and apply to mask
    H, _ = cv2.findHomography(gridcorners, imagecorners, cv2.RANSAC)
    mask = cv2.warpPerspective(grown_mask, H, (image.shape[1], image.shape[0]))
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_aligned_mask.jpg"), mask) 
    
    '''
    #second transformation to secure vertical alignment
    mask_h = get_h_mid(mask)
    image_h = get_h_mid(create_binary_mask(image))
    dy = get_midpoint(image_h)- get_midpoint(mask_h)
    if inspection:
        check_h_line(mask, mask_h)
        check_h_line(create_binary_mask(image), image_h)
    H, W = mask.shape[:2]
    M = np.float32([[1, 0, 0], [0, 1, dy]])  # translation matrix
    shifted_mask= cv2.warpAffine(mask, M, (W, H), borderValue=255)  # white background
    reset the following to shifted_mask
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_shifted_mask.jpg")) 
    '''

    #replace black background in image with yellow (background color) by using the mask
    yellow_mask = mask == 0
    image_wo_grid = image.copy()
    image_wo_grid[yellow_mask] = [0,255,255]
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_wo_grid.jpg"), image_wo_grid) 
    
    #crop the image
    x, y, w, h = cv2.boundingRect(imageYST)
    cropped_image_wo_grid = image_wo_grid[y:y+h, x:x+w]
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_cropped.jpg"), cropped_image_wo_grid) 
    
    #handcrafted features for filtering bounding boxes
    if "TRIAVA" in image_file:
        min_area = 100  
        max_area = 1000 
    else:
        min_area = 1000 
        max_area = 10000 

    #find bounding boxes, filtered by handcrafted features and ratio of w/h, scale them    
    if inspection: cv2.imwrite(os.path.join(test_folder, filename + "_binary_mask.jpg"), create_binary_mask(cropped_image_wo_grid)) 
    rectangles= get_list_of_rectangles(cropped_image_wo_grid, min_area, max_area, scale = 1.5, max_ratio = 2)

    ##changed from here
    #only for testing: draw the found boxes into the labeled image
    image_cropped = image[y:y+h, x:x+w]
    image_labels = draw_bounding_boxes(image_cropped, rectangles)
    cv2.imwrite(os.path.join(test_folder, filename + "_w_labels.jpg"), image_labels)
    
    
    
    #finding the rectangles given by the yolo labels
    label_file = os.path.splitext(image_file)[0] + ".txt"
    labels_path = "/user/christoph.wald/u15287/big-scratch/02_splitted_data/train_labeled/labels_uncropped"
    label_path = os.path.join(labels_path, label_file)
    with open(label_path, "r") as f:
        yolo_labels = f.read().splitlines()
    yolo_rectangles = yolo_labels_to_rectangles(yolo_labels, image.shape)
    cropped_yolo_rectangles = transform_rectangles_to_cropped(yolo_rectangles, x, y,  w,h)
    #image_labels = draw_bounding_boxes(image, yolo_rectangles, color = (0,255,0))
    #cv2.imwrite(os.path.join(test_folder, filename + "_w_labels.jpg"), image_labels)
    image_labels = draw_bounding_boxes(image_labels, cropped_yolo_rectangles, color = (0,255,0))
    cv2.imwrite(os.path.join(test_folder, filename + "_w_labels.jpg"), image_labels)
    rectangles = [(x, y, x + w, y + h) for (x, y, w, h) in rectangles]
    cropped_yolo_rectangles = [(x, y, x + w, y + h) for (x, y, w, h) in cropped_yolo_rectangles]
    entry.append(evaluate_detections(rectangles, cropped_yolo_rectangles))
    results.append(entry)

#evaluate
rows = []
for name, metrics in results:
    row = {"image": name, **metrics}  # merge the dict with image name
    rows.append(row)

df = pd.DataFrame(rows)
df['prefix'] = df['image'].str[:6]  # first 6 chars like LIRIBO, BRAIIM, TRIAVA

# Sum TP, FP, FN over each prefix
grouped = df.groupby('prefix')[['TP', 'FP', 'FN']].sum().reset_index()
grouped['precision'] = grouped['TP'] / (grouped['TP'] + grouped['FP'])
grouped['recall'] = grouped['TP'] / (grouped['TP'] + grouped['FN'])

TP_total = df['TP'].sum()
FP_total = df['FP'].sum()
FN_total = df['FN'].sum()

precision_overall = TP_total / (TP_total + FP_total)
recall_overall = TP_total / (TP_total + FN_total)

print(grouped)
print("Overall precision:", precision_overall)
print("Overall recall:", recall_overall)
grouped.to_csv("results_image_processing.csv")

Loading LIRIBO_0629.jpg.
26 bounding boxes found.
16 bounding boxes found.
Loading BRAIIM_0003.jpg.
19 bounding boxes found.
19 bounding boxes found.
Loading LIRIBO_0002.jpg.
23 bounding boxes found.
10 bounding boxes found.
Loading TRIAVA_0136.jpg.
14 bounding boxes found.
43 bounding boxes found.
Loading BRAIIM_0945.jpg.
18 bounding boxes found.
18 bounding boxes found.
   prefix    TP    FP    FN  precision    recall
0  BRAIIM  2578  2867  1905   0.473462  0.575061
1  LIRIBO  3338  4998  1594   0.400432  0.676805
2  TRIAVA  1121  2198  6819   0.337752  0.141184
Overall precision: 0.4115204678362573
Overall recall: 0.4054739268222414
